In [1]:
import pandas as pd

In [2]:
census=pd.read_csv("C:/Users/mohamed ismail/Desktop/census_data.csv")

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
def label_fix(label):
    if label ==' <=50K':
        return 0
    else:
        return 1

In [6]:
census['income_bracket']=census['income_bracket'].apply(label_fix)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

In [9]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [10]:
import tensorflow as tf

In [11]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [12]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [13]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

In [14]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [15]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\MOHAME~1\\AppData\\Local\\Temp\\tmpzhzi2x1e', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000005849F89080>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
model.train(input_fn=input_func,steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\MOHAME~1\AppData\Local\Temp\tmpzhzi2x1e\model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:global_step/sec: 122.621
INFO:tensorflow:loss = 647.613, step = 101 (0.822 sec)
INFO:tensorflow:global_step/sec: 189.122
INFO:tensorflow:loss = 117.397, step = 201 (0.527 sec)
INFO:tensorflow:global_step/sec: 206.337
INFO:tensorflow:loss = 777.547, step = 301 (0.484 sec)
INFO:tensorflow:global_step/sec: 202.715
INFO:tensorflow:loss = 113.566, step = 401 (0.493 sec)
INFO:tensorflow:global_step/sec: 198.8
INFO:tensorflow:loss = 37.2125, step = 501 (0.504 sec)
INFO:tensorflow:global_step/sec: 154.637
INFO:tensorflow:loss = 381.09, step = 601 (0.649 sec)
INFO:tensorflow:global_step/sec: 153.022
INFO:tensorflow:loss = 365.832, step = 701 (0.662 sec)
INFO:tensorflow:global_step/sec: 183.387
INFO:tensorflow:loss = 244.246, step = 801 (0.534 sec)
INFO:tensorflow:global_step/sec: 200.664
IN

In [17]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [18]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Restoring parameters from C:\Users\MOHAME~1\AppData\Local\Temp\tmpzhzi2x1e\model.ckpt-5000


In [19]:
predictions[0]

{'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'logistic': array([ 0.26217404], dtype=float32),
 'logits': array([-1.03469944], dtype=float32),
 'probabilities': array([ 0.73782599,  0.26217404], dtype=float32)}

In [20]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [21]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]